## init

In [13]:
%config InteractiveShell.ast_node_interactivity = 'last_expr_or_assign'

In [71]:
import sympy as sym
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, interact_manual

## n(t) for $\dot{V}=0$ (testing SymPy)

In [22]:
tau, n_inf, t = sym.symbols('τ, n_\infty, t')

In [23]:
n = sym.Function('n')(t)

n(t)

In [24]:
ndot = n.diff(t)

Derivative(n(t), t)

In [25]:
V_nullcline__n = sym.Eq(ndot, (n_inf - n)/tau)

Eq(Derivative(n(t), t), (n_\infty - n(t))/τ)

In [26]:
sym.dsolve(V_nullcline__n)

Eq(n(t), C1*exp(-t/τ) + n_\infty)

## n(V) for $\dot{V} = 0$

In [22]:
def plot(V_1_2, k, I, g_L, E_L, g_Na, E_Na, g_K, E_K):
    
    def m_inf(V):
        return 1/(1+np.exp((V_1_2-V)/k))

    def n(V):
        num = I - g_L*(V-E_L) - g_Na*m_inf(V)*(V-E_Na)
        denom = g_K*(V-E_K)
        return num/denom

    V = np.linspace(-100, 70, 1000)
    plt.plot(V, n(V))
    plt.xlabel('V (mV)')
    plt.ylabel('n')

Units:
- Voltages in mV
- Conductances in mS


In [24]:
interact_manual(
    plot,
    V_1_2 = 1.5,
    k = 16,
    I = 0,
    g_L = 19,
    E_L = -67,
    g_Na = 74,
    E_Na = 80,  #60
    g_K = 10,
    E_K = -120,  #-90
);

interactive(children=(FloatSlider(value=1.5, description='V_1_2', max=4.5, min=-1.5), IntSlider(value=16, desc…

## Simulate Izhikevich neuron

Units are pF, mV, ms, and pA.  
(a is in 1/ms).

In [63]:
def izh_neuron(C, k, v_r, v_t, v_peak, a, b, c, d, I):
    
    dv_dt = lambda v,u: (k*(v-v_r)*(v-v_t) - u + I)/C
    du_dt = lambda v,u: a*(b*(v-v_r) - u)

    v_t0 = v_r
    u_t0 = 0

    T = 1000
    dt = 0.1

    N = round(T/dt)  # number of simulation steps
    v = np.ones(N) * v_t0
    u = np.ones(N) * u_t0
    for i in range(N-1):
        v[i+1] = v[i] + dt * dv_dt(v[i], u[i])
        u[i+1] = u[i] + dt * du_dt(v[i], u[i])
        if v[i+1] >= v_peak:
            v[i] = v_peak
            v[i+1] = c
            u[i+1] = u[i+1] + d

    t = np.linspace(0, T, N)
    fig, ax = plt.subplots()
    ax.plot(t, v);
    ax.set(xlabel="t (ms)", ylabel="V (mV)", ylim=[-75, 30])

In [72]:
cortical_RS = dict(C=100, k=0.7, v_r=-60, v_t=-40, v_peak=35, a=0.03, b=-2, c=-50, d=100)
cortical_IB = dict(C=150, k=1.2, v_r=-75, v_t=-45, v_peak=35, a=0.01, b= 5, c=-56, d=130)
cortical_CH = dict(C= 50, k=1.5, v_r=-60, v_t=-40, v_peak=25, a=0.03, b= 1, c=-40, d=150)

interact(izh_neuron, I=60, **cortical_RS);

interactive(children=(IntSlider(value=100, description='C', max=300, min=-100), FloatSlider(value=0.7, descrip…